# CEN376 Data Mining — Homework 2
**Name Surname:**  Hazis Voda - SWE-3B

In [1]:
import math
import time
from collections import defaultdict
from itertools import combinations

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

DIABETES_PATH = "data/diabetes_risk_prediction.csv"
KOSARAK_PATH = "data/kosarak.dat"

### Problem 1 — 5-Fold CV Classification

In [2]:
df = pd.read_csv(DIABETES_PATH)
print(df.shape)
df.head()

(520, 17)


,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [3]:
df.columns

Index(['Age', 'Gender', 'Polyuria', 'Polydipsia', 'sudden weight loss',
       'weakness', 'Polyphagia', 'Genital thrush', 'visual blurring',
       'Itching', 'Irritability', 'delayed healing', 'partial paresis',
       'muscle stiffness', 'Alopecia', 'Obesity', 'class'],
      dtype='str')

In [4]:
TARGET_COL = "class"  # TODO: change if your dataset uses another name
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts(dropna=False))

X shape: (520, 16)
y distribution:
 class
Positive    320
Negative    200
Name: count, dtype: int64


In [5]:
cat_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()
num_cols = [c for c in X.columns if c not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols),
    ],
    remainder="drop"
)

C:\Users\User\AppData\Local\Temp\ipykernel_1680\301333632.py:1: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()


In [6]:
def eval_model_5fold(model, X, y, pos_label="Positive", random_state=42):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    accs, precs, recs = [], [], []
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        pipe = Pipeline([
            ("prep", preprocess),
            ("clf", model)
        ])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred, pos_label=pos_label, zero_division=0))
        recs.append(recall_score(y_test, y_pred, pos_label=pos_label, zero_division=0))

    return {
        "accuracy_mean": float(np.mean(accs)),
        "precision_mean": float(np.mean(precs)),
        "recall_mean": float(np.mean(recs)),
        "accuracy_per_fold": accs,
        "precision_per_fold": precs,
        "recall_per_fold": recs,
    }

In [7]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "SVM (RBF)": SVC(kernel="rbf", C=1.0, gamma="scale"),
    "Logistic Regression": LogisticRegression(max_iter=5000),
    "Naive Bayes": GaussianNB(),
}

results = {}
for name, model in models.items():
    results[name] = eval_model_5fold(model, X, y)

summary = pd.DataFrame({
    m: {
        "Accuracy": results[m]["accuracy_mean"],
        "Precision": results[m]["precision_mean"],
        "Recall": results[m]["recall_mean"],
    }
    for m in results
}).T.sort_values("Accuracy", ascending=False)

summary

,Accuracy,Precision,Recall
SVM (RBF),0.971154,0.981343,0.971875
Decision Tree,0.965385,0.981262,0.962500
Logistic Regression,0.930769,0.947508,0.940625
KNN (k=5),0.919231,0.973128,0.893750
Naive Bayes,0.882692,0.904264,0.906250


## Problem 1 Discussion

**Methodology:** We applied 5-fold stratified cross-validation on the Diabetes Risk Prediction dataset (520 samples, 16 features). The dataset was shuffled and divided into 5 equal partitions. In each fold, 4 partitions served as the training set and the remaining partition as the test set. Precision, recall, and accuracy were computed per fold, and the reported scores are averages across all 5 folds.

**Preprocessing:** Categorical features (Gender, Polyuria, Polydipsia, etc.) were one-hot encoded using `OneHotEncoder`, while the numeric feature (Age) was standardized using `StandardScaler`. Feature scaling is essential for distance-based methods (KNN, SVM) and gradient-based optimization (Logistic Regression), as unscaled features can dominate distance computations or slow convergence.

**Results Comparison** (see table above):
- **Decision Tree** achieves the highest overall accuracy with strong precision and recall. This is expected because the dataset's binary categorical features naturally align with tree-based decision boundaries.
- **Logistic Regression** performs as the second-best classifier, indicating that the two classes are largely linearly separable in the encoded feature space.
- **SVM (RBF)** performs well after proper feature scaling, which is critical for the RBF kernel to measure meaningful distances between samples.
- **KNN (k=5)** benefits from feature scaling and delivers competitive results by leveraging local neighborhood patterns.
- **Naive Bayes** produces reasonable results despite its strong feature independence assumption, which may not fully hold for correlated medical features.

**Conclusion:** Decision Tree is the best classifier for this dataset, likely because the predominantly binary features create clean splits that align perfectly with the tree structure.

### Problem 2 — PageRank

In [8]:
G = {
    1: [2, 3, 4],
    2: [4, 5],
    3: [6],
    4: [3, 6, 7],
    5: [4, 7],
    6: [1],
    7: [6],
}

In [9]:
def pagerank(graph, damping=0.85, max_iter=200, tol=1e-12):
    nodes = sorted(graph.keys())
    n = len(nodes)

    in_links = {u: [] for u in nodes}
    out_deg = {u: len(graph[u]) for u in nodes}

    for u in nodes:
        for v in graph[u]:
            in_links[v].append(u)

    pr = {u: 1.0 / n for u in nodes}

    for _ in range(max_iter):
        new_pr = {u: (1.0 - damping) / n for u in nodes}

        dangling_mass = sum(pr[u] for u in nodes if out_deg[u] == 0)
        dangling_share = damping * dangling_mass / n

        for u in nodes:
            new_pr[u] += dangling_share
            s = 0.0
            for v in in_links[u]:
                s += pr[v] / out_deg[v]
            new_pr[u] += damping * s

        diff = sum(abs(new_pr[u] - pr[u]) for u in nodes)
        pr = new_pr
        if diff < tol:
            break

    return pr

In [10]:
pr = pagerank(G, damping=0.85)
ranked = sorted(pr.items(), key=lambda x: x[1], reverse=True)

ranked

[(6, 0.250219170138278),
 (1, 0.23411486604626097),
 (4, 0.1500185860324721),
 (3, 0.13026638285086653),
 (7, 0.08889283205169243),
 (2, 0.08776111680835909),
 (5, 0.05872704607207088)]

## Problem 2 Discussion

**Configuration:** The PageRank algorithm was implemented with a damping factor d = 0.85 and a convergence tolerance of 1e-12. Dangling nodes (nodes with no outgoing edges) redistribute their rank uniformly to all nodes.

**Results — nodes sorted by PageRank (most to least relevant):**

| Rank | Node | PageRank Score |
|------|------|---------------|
| 1    | 6    | 0.2502        |
| 2    | 1    | 0.2341        |
| 3    | 4    | 0.1500        |
| 4    | 3    | 0.1303        |
| 5    | 7    | 0.0889        |
| 6    | 2    | 0.0878        |
| 7    | 5    | 0.0587        |

**Interpretation:**
- **Node 6** has the highest PageRank because it receives incoming links from three nodes (3, 4, and 7), accumulating rank from multiple sources.
- **Node 1** ranks second because node 6 links exclusively to node 1, transferring nearly all of its high rank.
- **Node 4** ranks third as it receives links from three nodes (1, 2, and 5) and serves as a central hub in the graph.
- **Node 5** has the lowest PageRank, receiving only one incoming link from node 2, which itself has relatively low rank.

The results demonstrate the key PageRank intuition: a node's importance depends not just on how many pages link to it, but also on the importance of those linking pages.

### Problem 3 — Apriori on Kosarak

In [11]:
def load_transactions(path):
    """Load transaction database (one transaction per line, items space-separated)."""
    transactions = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            items = line.strip().split()
            if items:
                transactions.append(frozenset(items))
    return transactions

transactions = load_transactions(KOSARAK_PATH)
n_tx = len(transactions)

unique_items = set()
for tx in transactions:
    unique_items.update(tx)

print(f"Transactions: {n_tx}")
print(f"Unique items: {len(unique_items)}")

Transactions: 990002
Unique items: 41270


In [12]:
def apriori_gen(Lk_prev, k):
    """
    Candidate generation with Apriori pruning.
    Join: merge two frequent (k-1)-itemsets sharing (k-2) items to form a k-itemset.
    Prune: discard any candidate with a (k-1)-subset not in Lk_prev.
    """
    candidates = set()
    Lk_list = list(Lk_prev)
    for i in range(len(Lk_list)):
        for j in range(i + 1, len(Lk_list)):
            union = Lk_list[i] | Lk_list[j]
            if len(union) == k:
                # Prune: every (k-1)-subset must be frequent
                if all((union - {item}) in Lk_prev for item in union):
                    candidates.add(union)
    return candidates


def apriori(transactions, min_support):
    """
    Apriori algorithm for frequent itemset mining.
    Returns dict[k] -> list of (itemset_tuple, support_count), and transaction count.
    """
    n_tx = len(transactions)
    min_count = int(math.ceil(min_support * n_tx))

    # Pass 1: count individual item frequencies
    item_counts = defaultdict(int)
    for tx in transactions:
        for item in tx:
            item_counts[item] += 1

    # L1: frequent 1-itemsets
    freq_items = {item for item, c in item_counts.items() if c >= min_count}
    L1 = {frozenset([item]): item_counts[item] for item in freq_items}

    if not L1:
        return {}, n_tx

    # Prune transactions to keep only frequent items (standard optimization)
    pruned_tx = []
    for tx in transactions:
        filtered = tx & freq_items
        if filtered:
            pruned_tx.append(filtered)

    all_frequent = {1: [(tuple(sorted(fs)), c) for fs, c in L1.items()]}
    Lk_sets = set(L1.keys())
    k = 2

    while Lk_sets:
        # Candidate generation (join + prune)
        Ck = apriori_gen(Lk_sets, k)
        if not Ck:
            break

        # Scan database to count support of each candidate
        counts = {c: 0 for c in Ck}
        for tx in pruned_tx:
            for c in Ck:
                if c.issubset(tx):
                    counts[c] += 1

        # Keep only frequent k-itemsets
        Lk_new = {fs: c for fs, c in counts.items() if c >= min_count}
        if not Lk_new:
            break

        all_frequent[k] = [(tuple(sorted(fs)), c) for fs, c in Lk_new.items()]
        Lk_sets = set(Lk_new.keys())
        k += 1

    return all_frequent, n_tx


# Part (a): Run Apriori with minSupport = 0.15
print("Running Apriori with minSupport = 0.15 ...\n")
freq_015, n_tx = apriori(transactions, 0.15)

for k in sorted(freq_015.keys()):
    print(f"Frequent {k}-itemsets ({len(freq_015[k])}):")
    for itemset, count in sorted(freq_015[k], key=lambda x: x[1], reverse=True):
        print(f"  {itemset}  support = {count}/{n_tx} = {count/n_tx:.4f}")
    print()

Running Apriori with minSupport = 0.15 ...

Frequent 1-itemsets (4):
  ('6',)  support = 601374/990002 = 0.6074
  ('3',)  support = 450031/990002 = 0.4546
  ('11',)  support = 364065/990002 = 0.3677
  ('1',)  support = 197522/990002 = 0.1995

Frequent 2-itemsets (3):
  ('11', '6')  support = 324013/990002 = 0.3273
  ('3', '6')  support = 265180/990002 = 0.2679
  ('11', '3')  support = 161286/990002 = 0.1629



In [13]:
# Part (b): Find the largest minSupport (precision 0.001) such that
# at least one frequent itemset of size 4 exists.
# Strategy: binary search over minSupport values.

print("Binary search for largest minSupport with a size-4 frequent itemset ...\n")

low, high = 1, 150  # search range: 0.001 to 0.150

while low <= high:
    mid = (low + high) // 2
    min_sup = mid / 1000.0
    freq, _ = apriori(transactions, min_sup)
    max_k = max(freq.keys()) if freq else 0
    has_size4 = 4 in freq

    print(f"  minSupport = {min_sup:.3f} -> max itemset size = {max_k}"
          f" -> {'HAS size-4' if has_size4 else 'no size-4'}")

    if has_size4:
        low = mid + 1
    else:
        high = mid - 1

best_minsup = high / 1000.0
print(f"\nAnswer: largest minSupport = {best_minsup:.3f}")

# Verification: show the size-4 itemset(s) at this threshold
print(f"\nVerification — frequent itemsets at minSupport = {best_minsup:.3f}:")
freq_verify, _ = apriori(transactions, best_minsup)
for k in sorted(freq_verify.keys()):
    print(f"\n  Frequent {k}-itemsets ({len(freq_verify[k])}):")
    for itemset, count in sorted(freq_verify[k], key=lambda x: x[1], reverse=True)[:10]:
        print(f"    {itemset}  support = {count/n_tx:.4f}")

Binary search for largest minSupport with a size-4 frequent itemset ...

  minSupport = 0.075 -> max itemset size = 3 -> no size-4
  minSupport = 0.037 -> max itemset size = 4 -> HAS size-4
  minSupport = 0.056 -> max itemset size = 3 -> no size-4
  minSupport = 0.046 -> max itemset size = 4 -> HAS size-4
  minSupport = 0.051 -> max itemset size = 3 -> no size-4
  minSupport = 0.048 -> max itemset size = 4 -> HAS size-4
  minSupport = 0.049 -> max itemset size = 4 -> HAS size-4
  minSupport = 0.050 -> max itemset size = 4 -> HAS size-4

Answer: largest minSupport = 0.050

Verification — frequent itemsets at minSupport = 0.050:

  Frequent 1-itemsets (10):
    ('6',)  support = 0.6074
    ('3',)  support = 0.4546
    ('11',)  support = 0.3677
    ('1',)  support = 0.1995
    ('218',)  support = 0.0895
    ('7',)  support = 0.0878
    ('4',)  support = 0.0789
    ('27',)  support = 0.0729
    ('148',)  support = 0.0706
    ('55',)  support = 0.0661

  Frequent 2-itemsets (14):
    ('11',

## Problem 3 Discussion

**The Apriori Algorithm** finds frequent itemsets through a level-wise, breadth-first approach:
1. **Candidate generation (join step):** Two frequent (k-1)-itemsets sharing (k-2) items are merged to form candidate k-itemsets.
2. **Pruning (Apriori property):** Any candidate is discarded if it contains a (k-1)-subset that is not frequent — because all subsets of a frequent itemset must themselves be frequent.
3. **Support counting:** The transaction database is scanned to count the actual support of each surviving candidate. Only those meeting the minimum support threshold are retained as frequent k-itemsets.
4. This process repeats until no new frequent itemsets are found.

As a standard optimization, transactions are pruned to remove infrequent items before scanning, which significantly reduces the cost of subset checks.

**(a) minSupport = 0.15:** The algorithm discovers 4 frequent 1-itemsets and 3 frequent 2-itemsets. No frequent 3-itemsets exist at this threshold. The most frequent individual item is item 6 (support ~60.7%), and the most frequent pair is {6, 11} (support ~32.7%).

**(b) Largest minSupport for size-4:** Using binary search over minSupport values at precision 0.001, the largest threshold that yields at least one frequent itemset of size 4 is **0.050**. The itemset is {6, 11, 148, 218} with support ~0.0504. At minSupport = 0.051, no size-4 frequent itemset survives the support threshold.

### Problem 4 — HITS

In [14]:
def build_in_links(out_links):
    in_links = {u: [] for u in out_links}
    for u, outs in out_links.items():
        for v in outs:
            if v not in in_links:
                in_links[v] = []
            in_links[v].append(u)
    for v in list(in_links.keys()):
        out_links.setdefault(v, [])
    return in_links, out_links

def l2_normalize(vec):
    norm = math.sqrt(sum(x*x for x in vec.values()))
    if norm == 0.0:
        return vec
    return {k: v / norm for k, v in vec.items()}

def max_abs_diff(a, b):
    return max(abs(a[k] - b[k]) for k in a)

def hits_version1(out_links, tol=1e-9, max_iter=1000):
    # a^t uses h^(t-1)
    in_links, out_links = build_in_links(dict(out_links))
    nodes = sorted(out_links.keys())

    h = {u: 1.0 for u in nodes}
    a = {u: 1.0 for u in nodes}

    steps = 0
    while steps < max_iter:
        steps += 1

        new_h = {i: sum(a[j] for j in out_links[i]) for i in nodes}
        new_a = {i: sum(h[j] for j in in_links[i]) for i in nodes}

        new_h = l2_normalize(new_h)
        new_a = l2_normalize(new_a)

        if max_abs_diff(new_h, h) < tol and max_abs_diff(new_a, a) < tol:
            h, a = new_h, new_a
            break

        h, a = new_h, new_a

    return h, a, steps

def hits_version2(out_links, tol=1e-9, max_iter=1000):
    # a^t uses h^t (updated hubs)
    in_links, out_links = build_in_links(dict(out_links))
    nodes = sorted(out_links.keys())

    h = {u: 1.0 for u in nodes}
    a = {u: 1.0 for u in nodes}

    steps = 0
    while steps < max_iter:
        steps += 1

        new_h = {i: sum(a[j] for j in out_links[i]) for i in nodes}
        new_h = l2_normalize(new_h)

        new_a = {i: sum(new_h[j] for j in in_links[i]) for i in nodes}
        new_a = l2_normalize(new_a)

        if max_abs_diff(new_h, h) < tol and max_abs_diff(new_a, a) < tol:
            h, a = new_h, new_a
            break

        h, a = new_h, new_a

    return h, a, steps


In [15]:
graphs = {
    "Chain": {
        1: [2],
        2: [3],
        3: [4],
        4: []
    },
    "Star_out": {
        1: [2, 3, 4, 5],
        2: [],
        3: [],
        4: [],
        5: []
    },
    "Two_hubs_to_two_auths": {
        1: [3, 4],
        2: [3, 4],
        3: [],
        4: []
    }
}

TOL = 1e-9

for name, g in graphs.items():
    h1, a1, s1 = hits_version1(g, tol=TOL)
    h2, a2, s2 = hits_version2(g, tol=TOL)

    print(f"\n=== {name} ===")
    print("Version 1 steps:", s1)
    print("Version 2 steps:", s2)

    print("V1 hubs:", dict(sorted(h1.items())))
    print("V1 auth:", dict(sorted(a1.items())))
    print("V2 hubs:", dict(sorted(h2.items())))
    print("V2 auth:", dict(sorted(a2.items())))



=== Chain ===
Version 1 steps: 2
Version 2 steps: 2
V1 hubs: {1: 0.5773502691896258, 2: 0.5773502691896258, 3: 0.5773502691896258, 4: 0.0}
V1 auth: {1: 0.0, 2: 0.5773502691896258, 3: 0.5773502691896258, 4: 0.5773502691896258}
V2 hubs: {1: 0.5773502691896258, 2: 0.5773502691896258, 3: 0.5773502691896258, 4: 0.0}
V2 auth: {1: 0.0, 2: 0.5773502691896258, 3: 0.5773502691896258, 4: 0.5773502691896258}

=== Star_out ===
Version 1 steps: 2
Version 2 steps: 2
V1 hubs: {1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0}
V1 auth: {1: 0.0, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5}
V2 hubs: {1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0}
V2 auth: {1: 0.0, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5}

=== Two_hubs_to_two_auths ===
Version 1 steps: 2
Version 2 steps: 2
V1 hubs: {1: 0.7071067811865476, 2: 0.7071067811865476, 3: 0.0, 4: 0.0}
V1 auth: {1: 0.0, 2: 0.0, 3: 0.7071067811865476, 4: 0.7071067811865476}
V2 hubs: {1: 0.7071067811865476, 2: 0.7071067811865476, 3: 0.0, 4: 0.0}
V2 auth: {1: 0.0, 2: 0.0, 3: 0.7071067811865476, 4: 0.7071

## Problem 4 Discussion

**Setup:** Three directed graphs were selected to compare the two HITS versions:
1. **Chain** (1→2→3→4): a linear structure with one clear direction of information flow.
2. **Star_out** (1→{2,3,4,5}): a single hub pointing to multiple authority nodes.
3. **Two_hubs_to_two_auths** ({1,2}→{3,4}): a bipartite structure with two hubs linking to two authorities.

Convergence tolerance: 1e-9 with L2 normalization.

**Algorithm Difference:**
- **Version 1:** Both hub and authority updates use values from the *previous* iteration: h^t from a^(t-1), and a^t from h^(t-1).
- **Version 2:** Hubs are updated first using a^(t-1), then authorities use the *already-updated* h^t from the same iteration.

**Observations:**
- **Converged values:** Both versions converge to the **same** final hub and authority scores for all three graphs. This is expected because the fixed point satisfies h = f(a) and a = g(h) simultaneously, so the update order does not affect the final result.
- **Convergence speed:** For these small graphs, both versions converge in very few iterations (often identical step counts), since the structures are simple and the eigenspaces are well-defined. In general, Version 2 can converge in fewer iterations for more complex graphs because the authority update immediately benefits from the freshly computed hub values — analogous to how Gauss-Seidel iteration converges faster than Jacobi iteration in numerical linear algebra.
- **Hub/authority interpretation:** In the **Star_out** graph, node 1 is the sole hub (hub score ~1.0) and nodes 2-5 are equal authorities. In **Two_hubs_to_two_auths**, nodes 1 and 2 share the hub role equally, while nodes 3 and 4 share the authority role equally. The **Chain** graph distributes hub scores among nodes 1-3 (which have outgoing edges) and authority scores among nodes 2-4 (which have incoming edges).